### Your Task: Create a Cubic Spline with Custom End Conditions

1. Create a cubic spline for the bridge data using the 'natural' end condition
2. Create another spline with custom end slopes: left slope = -8, right slope = 8
3. Create a third spline using the 'not-a-knot' condition
4. Plot all three splines on the same graph for comparison
5. Extract and print the coefficients for the segment between x=15 and x=20

**Note**: Remember that the syntax for specifying a clamped spline is: `bc_type=((1, left_slope), (1, right_slope))`

In [ ]:
# Your code here
# 1. Create splines with different end conditions

# 2. Plot the splines

# 3. Find and print coefficients for the segment between x=15 and x=20

## Task 6: Discussion Questions

After completing the above tasks, answer the following questions:

1. How do the different end conditions affect the behavior of the spline near the endpoints? Which end condition seems most appropriate for the bridge girder deflection problem?

2. When examining the coefficients for the different splines, what differences do you notice?

3. In which engineering scenarios would you prefer to use a natural spline versus a clamped spline?

4. What are the advantages and disadvantages of spline interpolation compared to other interpolation methods (like polynomial interpolation)?

Your answers here:

1. 

2. 

3. 

4. 

## Conclusion

In this worksheet, we've explored the fundamentals of cubic spline interpolation using SciPy. We've learned how to:

1. Create cubic splines with various end conditions
2. Visualize the resulting interpolations
3. Extract and understand the spline coefficients
4. Apply splines to engineering problems like beam deflection analysis

Spline interpolation is a powerful tool for engineering applications where smooth interpolation of measured data is essential. By understanding how to work with end conditions and coefficients, you can create accurate and physically meaningful interpolations.

### Further Reading

- [SciPy Documentation on `CubicSpline`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html)
- [SciPy Tutorial on Interpolation](https://docs.scipy.org/doc/scipy/tutorial/interpolate.html)
- _Numerical Methods for Engineers_, by Steven C. Chapra and Raymond P. Canale - Chapter on Spline Interpolation
- _Applied Numerical Methods with MATLAB_, by Steven C. Chapra - Section on Splines and Piecewise Interpolation